# 03 - Stage 3: LLM Judge with LoRA Fine-Tuning

## Overview
This notebook implements Stage 3 - the final scoring layer with explainability:
- **Stage 3: LLM Judge with Structured Outputs**
- Load Llama-3.2-1B or Mistral-7B with 4-bit quantization
- Fine-tune with LoRA for resume-JD matching
- Generate structured JSON outputs with scores + explanations
- Provide human-readable justifications for hiring decisions

**⚠️ GPU REQUIRED**: This notebook needs at least T4 GPU (15GB VRAM)

**Key Features**:
- 🧠 Explainable AI: Not just scores, but reasoning
- 💾 Memory efficient: 4-bit quantization + LoRA (no full model fine-tuning)
- 📋 Structured outputs: Force JSON format for parsing
- 🎯 Domain adaptation: Fine-tune on resume screening task

**Estimated Time**: 2-4 hours (depending on GPU)

## 1. Environment Setup & GPU Check

In [ ]:
# CRITICAL: Check GPU availability
import subprocess
import sys
import os

IN_COLAB = 'google.colab' in sys.modules
IN_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ

print(f"Running in Google Colab: {IN_COLAB}")
print(f"Running in Kaggle: {IN_KAGGLE}")

# Check GPU
print("\n" + "="*80)
print("GPU AVAILABILITY CHECK")
print("="*80)

try:
    gpu_info = subprocess.check_output(['nvidia-smi'], encoding='utf-8')
    print(gpu_info)
    print("\n✅ GPU detected! Proceeding with fine-tuning...")
except:
    print("\n❌ NO GPU DETECTED!")
    print("\nThis notebook REQUIRES a GPU. Please enable GPU:")
    if IN_COLAB:
        print("  1. Runtime → Change runtime type")
        print("  2. Select 'T4 GPU' or 'A100 GPU'")
        print("  3. Click 'Save' and restart")
    elif IN_KAGGLE:
        print("  1. Settings → Accelerator")
        print("  2. Select 'GPU T4 x2'")
        print("  3. Restart session")
    else:
        print("  Ensure CUDA-capable GPU is available")
    
    print("\n⚠️ Continuing anyway for demonstration (will fail at training)")

## 2. Install Required Packages

In [ ]:
%%capture
# Install transformers ecosystem
!pip install -U transformers accelerate peft
!pip install -U bitsandbytes  # 4-bit quantization
!pip install -U trl  # Transformer Reinforcement Learning (SFTTrainer)
!pip install -U datasets

# Optional: Weights & Biases for experiment tracking
!pip install -U wandb

# Utilities
!pip install pandas numpy tqdm

In [ ]:
# Import libraries
import torch
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModel
)
from trl import SFTTrainer
from datasets import Dataset

import pandas as pd
import numpy as np
import json
import pickle
from pathlib import Path
from typing import List, Dict
import warnings
from tqdm.auto import tqdm

warnings.filterwarnings('ignore')

print(f"✅ torch version: {torch.__version__}")
print(f"✅ transformers version: {transformers.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 3. Load Configuration and Stage 2 Results

In [ ]:
# Setup paths
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_PATH = Path('/content/drive/MyDrive/resume_screening_project')
elif IN_KAGGLE:
    BASE_PATH = Path('/kaggle/working/resume_screening_project')
else:
    BASE_PATH = Path('./resume_screening_project')

MODELS_PATH = BASE_PATH / 'models'
STAGE2_PATH = MODELS_PATH / 'stage2_reranker'
STAGE3_PATH = MODELS_PATH / 'stage3_llm_judge'
OUTPUTS_PATH = BASE_PATH / 'outputs'

STAGE3_PATH.mkdir(parents=True, exist_ok=True)

print(f"📁 Working Directory: {BASE_PATH}")

In [ ]:
# Load Stage 2 results
cache_path = STAGE2_PATH / 'reranking_cache.pkl'

if cache_path.exists():
    print("Loading Stage 2 re-ranking results...")
    with open(cache_path, 'rb') as f:
        stage2_cache = pickle.load(f)
    
    job_descriptions = stage2_cache['job_descriptions']
    stage2_results = stage2_cache['reranked_results']
    
    print(f"✅ Loaded Stage 2 results")
    print(f"   - Job descriptions: {len(job_descriptions)}")
    print(f"   - Top candidates per JD: {len(stage2_results[0])}")
else:
    print("⚠️ Stage 2 cache not found. Creating sample data...")
    job_descriptions = ["ML Engineer with Python experience"]
    stage2_results = [[{
        'resume_text': f'Candidate {i} with ML skills',
        'stage2_score': 0.8 - i*0.01
    } for i in range(10)]]

## 4. Prepare Training Data

Format: Chat template with system, user, and assistant messages

In [ ]:
# Prepare training dataset
def create_training_examples(jd_list, candidates_list, max_examples=500):
    """
    Create training examples in chat format.
    """
    examples = []
    
    for jd, candidates in zip(jd_list, candidates_list):
        # Take top-5 (positive examples) and bottom-5 (negative examples)
        top_candidates = candidates[:5]
        bottom_candidates = candidates[-5:]
        
        for cand_list, is_positive in [(top_candidates, True), (bottom_candidates, False)]:
            for cand in cand_list:
                # Create score and explanation
                score = int(cand.get('stage2_score', 0.5) * 100)
                
                if is_positive:
                    explanation = f"Strong match. Candidate demonstrates relevant skills and experience aligned with job requirements. Score: {score}/100."
                    strengths = "Technical skills, experience level, domain expertise"
                    gaps = "Minor: Could improve in specific areas"
                else:
                    explanation = f"Weak match. Significant gaps in required qualifications. Score: {score}/100."
                    strengths = "Some transferable skills"
                    gaps = "Major: Lacking core requirements"
                
                # Format as chat
                conversation = [
                    {"role": "system", "content": "You are an expert technical recruiter. Analyze the job description and resume, then provide a match score (0-100) with detailed justification in JSON format."},
                    {"role": "user", "content": f"Job Description:\n{jd[:500]}\n\nResume:\n{cand['resume_text'][:500]}\n\nProvide match analysis."},
                    {"role": "assistant", "content": json.dumps({
                        "match_score": score,
                        "explanation": explanation,
                        "key_strengths": strengths,
                        "gaps": gaps,
                        "recommendation": "Strong candidate" if is_positive else "Not recommended"
                    }, indent=2)}
                ]
                
                examples.append({"messages": conversation})
                
                if len(examples) >= max_examples:
                    return examples
    
    return examples

print("Creating training dataset...")
training_examples = create_training_examples(job_descriptions, stage2_results, max_examples=500)

print(f"✅ Created {len(training_examples)} training examples")
print(f"\nExample format:")
print(json.dumps(training_examples[0], indent=2)[:500] + "...")

In [ ]:
# Convert to HuggingFace Dataset
dataset = Dataset.from_list(training_examples)

# Train/eval split
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = dataset['train']
eval_dataset = dataset['test']

print(f"Dataset splits:")
print(f"  Train: {len(train_dataset)} examples")
print(f"  Eval:  {len(eval_dataset)} examples")

## 5. Load Base Model with 4-bit Quantization

Using Llama-3.2-1B for speed (or Mistral-7B for quality)

In [ ]:
# Model selection (choose based on VRAM)
# MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"  # Requires HF access token
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Open alternative
# MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"  # Better quality, needs 16GB+ VRAM

print(f"Loading model: {MODEL_NAME}")
print("This may take a few minutes...\n")

In [ ]:
# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"✅ Model loaded")
print(f"   Model size: {model.get_memory_footprint() / 1024**3:.2f} GB")
print(f"   Vocab size: {len(tokenizer)}")

## 6. Setup LoRA Configuration

In [ ]:
# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# LoRA config
lora_config = LoraConfig(
    r=16,  # LoRA rank
    lora_alpha=32,  # Scaling factor
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA
model = get_peft_model(model, lora_config)

# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"✅ LoRA applied")
print(f"   Trainable params: {trainable_params:,} ({trainable_params/total_params*100:.2f}%)")
print(f"   Total params: {total_params:,}")
print(f"\n💡 Only {trainable_params/total_params*100:.2f}% of model is being fine-tuned!")

## 7. Training Configuration

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir=str(STAGE3_PATH / 'checkpoints'),
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,  # Effective batch size = 16
    learning_rate=2e-4,
    warmup_steps=100,
    logging_steps=10,
    save_steps=100,
    eval_steps=100,
    evaluation_strategy="steps",
    save_total_limit=3,
    fp16=True,  # Mixed precision
    optim="paged_adamw_8bit",  # Memory efficient optimizer
    load_best_model_at_end=True,
    report_to="none",  # Change to "wandb" if using W&B
)

print("Training configuration:")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Total steps: ~{len(train_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * training_args.num_train_epochs}")

## 8. Initialize SFTTrainer

In [ ]:
# Format function for chat template
def formatting_func(example):
    """Format messages into text for training."""
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
    )
    return text

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    formatting_func=formatting_func,
    max_seq_length=1024,
)

print("✅ Trainer initialized")

## 9. Fine-Tune Model

⏰ This will take 2-4 hours depending on GPU

In [ ]:
# Clear CUDA cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("Starting fine-tuning...")
print("This will take 2-4 hours. Monitor GPU memory in another terminal with: watch -n 1 nvidia-smi")
print("\n" + "="*80)

# Train!
trainer.train()

print("\n" + "="*80)
print("✅ Fine-tuning complete!")

## 10. Save LoRA Adapters

In [ ]:
# Save LoRA adapters (only ~50MB!)
lora_path = STAGE3_PATH / 'lora_adapters'
model.save_pretrained(lora_path)
tokenizer.save_pretrained(lora_path)

print(f"✅ LoRA adapters saved to: {lora_path}")

# Check size
import os
size_mb = sum(f.stat().st_size for f in lora_path.rglob('*') if f.is_file()) / 1024**2
print(f"   Size: {size_mb:.1f} MB (vs {model.get_memory_footprint() / 1024**2:.0f} MB for full model)")

## 11. Test Inference

In [ ]:
# Test inference function
def generate_match_analysis(jd: str, resume: str, model, tokenizer, max_new_tokens=512):
    """
    Generate structured match analysis.
    """
    messages = [
        {"role": "system", "content": "You are an expert technical recruiter. Analyze the job description and resume, then provide a match score (0-100) with detailed justification in JSON format."},
        {"role": "user", "content": f"Job Description:\n{jd[:500]}\n\nResume:\n{resume[:500]}\n\nProvide match analysis in JSON format with keys: match_score, explanation, key_strengths, gaps, recommendation."}
    ]
    
    # Format
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Decode
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract JSON (simple approach)
    try:
        json_start = response.find('{')
        json_end = response.rfind('}') + 1
        if json_start != -1 and json_end > json_start:
            json_str = response[json_start:json_end]
            return json.loads(json_str)
    except:
        pass
    
    return {"raw_response": response}

print("✅ Inference function defined")

In [ ]:
# Test on sample
test_jd = job_descriptions[0]
test_resume = stage2_results[0][0]['resume_text']

print("Testing inference...\n")
print("JD:", test_jd[:200], "...\n")
print("Resume:", test_resume[:200], "...\n")
print("="*80)

result = generate_match_analysis(test_jd, test_resume, model, tokenizer)

print("\nLLM Output:")
print(json.dumps(result, indent=2))

## 12. Batch Scoring

In [ ]:
# Score top-10 candidates for each JD
print("Generating LLM explanations for top candidates...\n")

llm_results = []

for jd, candidates in tqdm(zip(job_descriptions, stage2_results), total=len(job_descriptions)):
    jd_results = []
    
    for cand in candidates[:10]:  # Top 10 only
        analysis = generate_match_analysis(jd, cand['resume_text'], model, tokenizer)
        
        jd_results.append({
            **cand,
            'llm_analysis': analysis
        })
    
    llm_results.append(jd_results)

print(f"\n✅ Generated LLM explanations for {len(llm_results)} JDs")

In [ ]:
# Save results
stage3_cache = {
    'job_descriptions': job_descriptions,
    'llm_results': llm_results,
    'model_name': MODEL_NAME,
    'timestamp': pd.Timestamp.now().isoformat(),
}

cache_path = STAGE3_PATH / 'llm_results_cache.pkl'
with open(cache_path, 'wb') as f:
    pickle.dump(stage3_cache, f)

print(f"💾 Stage 3 results saved to: {cache_path}")

## 13. Summary

In [ ]:
print("="*80)
print(" " * 20 + "STAGE 3: LLM JUDGE FINE-TUNING COMPLETE")
print("="*80)

print("\n📊 Summary:")
print(f"   - Base model: {MODEL_NAME}")
print(f"   - Training examples: {len(train_dataset)}")
print(f"   - LoRA rank: {lora_config.r}")
print(f"   - Trainable params: {trainable_params/total_params*100:.2f}%")

print("\n💾 Saved Artifacts:")
print(f"   - LoRA adapters: {lora_path}")
print(f"   - LLM results: {cache_path.name}")
print(f"   - Checkpoints: {STAGE3_PATH / 'checkpoints'}")

print("\n🎯 Key Features:")
print("   ✓ Explainable AI: Scores with detailed reasoning")
print("   ✓ Structured outputs: JSON format for parsing")
print("   ✓ Memory efficient: 4-bit quantization + LoRA")
print("   ✓ Domain adapted: Fine-tuned on resume screening")

print("\n✅ Ready for Stage 4: Full Pipeline Integration")
print("   👉 Open: 04_full_pipeline_integration.ipynb")
print("="*80)